## Paper 1 Model 1: <br> Prediction of triaxial behavior of recycled aggregate concrete using multivariable regression

### Jinjun Xu, Yuliang Chen, Tianyu Xie, Xinyu Zhao, Beibei Xiong, Zongping Chen

#### **Objective**
A cost-effective design of concrete elements in compliance with sustainability principles requires to accurately predict the behavior of concrete under various loading conditions. However, approaches for the prediction and evaluation of recycled aggregate concretes (RAC) under triaxial load are very
scarce, which may hinder the transition of such an eco-friendly concrete from lab-based technique to widespread applications. 

As a first attempt to iron out this issue, this study develops some reliable and
accurate analytical tools on the basis of an experimental database containing the results of 193 cylindrical peak stress under axisymmetric triaxial load, 55 cubic peak stress under true triaxial load, 76 peak strains and 61 elastic moduli of RAC retrieved from an extensive review of the literature. 

First, a new empirical design-oriented model is presented refining the generic form of conventional confinement models for natural aggregate concrete while considering the impacts of RCAs. Then Grey Correlation Analysis (GCA) is conducted to look into the sensitivity of the key parameters that affect the triaxial behavior of RAC. The results of the GCA indicate that the triaxial behavior of RAC mainly depends on the effective water-to-cement binder ratio, aggregate-to-cement ratio, lateral stress conditions, exposure temperature, as well as the RCA replacement ratio. 

Having the input parameters identified by the GCA, three robust data-mining mathematic tools, namely Multiple Nonlinear Regression (MNR), Artificial
Neural Network (ANN), and hybrid Genetic Algorithm Artificial Neural Network (GA-ANN) are employed to simulate the mechanical performances of RAC under triaxial load.

**This notebook is modelling the Multiple Nonlinear Regression (MNR)**

In [ ]:
!pip install foundry-ml
!pip install pyyaml==5.4.1
!pip install mastml

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.1/51.1 KB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.1/211.1 KB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.6/51.6 KB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.3/155.3 KB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.3/111.3 KB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.8/95.8 KB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 KB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 KB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 8.9 MB/s eta 0:00:00
  Attempting uninstall: jsonschema
    Found existing installation: jsonschema 4.3.3
    Uninstall

In [14]:
from mastml.mastml import Mastml
from mastml.datasets import LocalDatasets
from mastml.preprocessing import SklearnPreprocessor
from mastml.models import SklearnModel, EnsembleModel
from mastml.feature_generators import PolynomialFeatureGenerator
from mastml.feature_selectors import NoSelect
import pandas as pd
import numpy as np
import os
import shutil
import sys

Figshare is an optional dependency. To import data from figshare, manually install figshare via git clone of git clone https://github.com/cognoma/figshare.git
scikit-lego is an optional dependency, enabling use of the LowessRegression model. If you want to use this model, do "pip install scikit-lego"


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
sys.path.append('/content/drive/MyDrive/University/2022-23 Senior/Research/MNR_Model')
from data_splitters import NoSplit

forestci is an optional dependency. To install latest forestci compatabilty with scikit-learn>=0.24, run pip install git+git://github.com/scikit-learn-contrib/forest-confidence-interval.git


#### **Experimental database, existing empirical models and new design-oriented model**

**Construction of experimental database**

dataA1 comprises of experimental data for RAC under axisymetric triaxial compressions <br>

Reference:

H.F. Yang, S.P. Meng, Z.H. Deng, Stress-strain curves of high-performance
recycled concrete under conventional triaxial compression, J. Jiangsu Univ.
(Nat. Sci. Ed.) 32 (5) (2011) 597–601. <br><br>
P. Folino, H. Xargay, Recycled aggregate concrete–mechanical behavior
under uniaxial and triaxial compression, Constr. Build. Mater. 56 (3) (2014)
21–31. <br><br>
Z.P. Chen, Y.L. Chen, J.J. Xu, J.Y. Xue, Experimental study on failure criterion
and stress-strain constitutive equation of recycled coarse aggregate concretes
under multiaxial compression, China Civ. Eng. J. 48 (12) (2015) 23–33. <br><br>
E.C. Meng, Y.L. Yu, J. Yuan, K. Qiao, Y.S. Su, Triaxial compressive strength
experiment study of recycled aggregate concrete after high temperatures,
Constr. Build. Mater. 155 (2017) 542–549.

In [ ]:
dataA1 = LocalDatasets._import(LocalDatasets('/content/drive/MyDrive/University/2022-23 Senior/Research/MNR_Model/dataset_tableA1.csv'))
dataA1['T[deg.C]'] = dataA1['T[deg.C]'].replace(np.nan, 25)
dataA1 = dataA1[['Specimen_Shape', 'D[mm]', 'H[mm]', 'RCA%', 'w_eff/c', 'a/c', 'S_3[MPa]', 'T[deg.C]', 'r_adr[kg/m3]', 'r_adn[kg/m3]', 'r_bdr[kg/m3]', 'r_bdn[kg/m3]', 'WA_RCA[%]', 'WA_NA[%]', 'CI_RCA[%]', 'CI_NA[%]', 'f_c[MPa]', 'e0[10^-3]', 'Ec[GPa]', 's_1[MPa]', 'e1[10^-3]']]
dataA1

,Specimen_Shape,D[mm],H[mm],RCA%,w_eff/c,a/c,S_3[MPa],T[deg.C],r_adr[kg/m3],r_adn[kg/m3],...,r_bdn[kg/m3],WA_RCA[%],WA_NA[%],CI_RCA[%],CI_NA[%],f_c[MPa],e0[10^-3],Ec[GPa],s_1[MPa],e1[10^-3]
0,C1,50,100,0,0.40,2.50,0.0,25.0,NaN,2760.0,...,1429.0,NaN,1.35,NaN,13.0,64.36,2.179,NaN,64.36,2.179
1,C1,50,100,0,0.40,2.50,8.0,25.0,NaN,2760.0,...,1429.0,NaN,1.35,NaN,13.0,64.36,2.179,NaN,120.67,10.202
2,C1,50,100,0,0.40,2.50,16.0,25.0,NaN,2760.0,...,1429.0,NaN,1.35,NaN,13.0,64.36,2.179,NaN,158.98,15.889
3,C1,50,100,0,0.40,2.50,24.0,25.0,NaN,2760.0,...,1429.0,NaN,1.35,NaN,13.0,64.36,2.179,NaN,188.16,19.157
4,C1,50,100,0,0.40,2.50,32.0,25.0,NaN,2760.0,...,1429.0,NaN,1.35,NaN,13.0,64.36,2.179,NaN,219.04,24.846
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
182,C2,100,200,30,0.41,2.15,20.0,500.0,2359.0,2652.0,...,1523.0,6.5,0.88,NaN,NaN,7.84,NaN,NaN,107.83,NaN
183,C2,100,200,70,0.41,2.15,0.0,500.0,2359.0,2652.0,...,1523.0,6.5,0.88,NaN,NaN,7.31,NaN,NaN,7.31,NaN
184,C2,100,200,70,0.41,2.15,5.0,500.0,2359.0,2652.0,...,1523.0,6.5,0.88,NaN,NaN,7.31,NaN,NaN,53.56,NaN
185,C2,100,200,70,0.41,2.15,10.0,500.0,2359.0,2652.0,...,1523.0,6.5,0.88,NaN,NaN,7.31,NaN,NaN,76.07,NaN


dataA2 comprises of experimental data for RAC under true triaxial compressions

Z.J. He, J.X. Zhang, Strength characteristics and failure criterion of plain
recycled aggregate concrete under triaxial stress states, Constr. Build. Mater.
54 (11) (2014) 354–362. <br><br>
Z.J. He, W.L. Cao, J.X. Zhang, L. Wang, Multiaxial mechanical properties of
plain recycled aggregate concrete, Mag. Concr. Res. 67 (8) (2015) 1–13. <br><br>
Z.H. Deng, Y.M. Wang, J. Sheng, X. Hu, Strength and deformation of recycled
aggregate concrete under triaxial compression, Constr. Build. Mater. 156
(2017) 1043–1052.


In [ ]:
dataA2 = LocalDatasets._import(LocalDatasets('/content/drive/MyDrive/University/2022-23 Senior/Research/MNR_Model/dataset_tableA2.csv'))
dataA2 = dataA2[['Specimen_Type', 'B[mm]', 'H[mm]', 'RCA%', 's2_to_s1', 's3_to_s1', 'W_eff/c', 'a/c', 'r_adr[kg/m3]', 'r_adn[kg/m3]', 'r_bdr[kg/m3]', 'r_bdn[kg/m3]', 'WA_RCA[%]', 'WA_NA[%]', 'CI_RCA[%]', 'CI_NA[%]', 'f_c[MPa]', 'e0[10^-3]', 'Ec[GPa]', 's_1[MPa]', 'e1[10^-3]']]
dataA2

,Specimen_Type,B[mm],H[mm],RCA%,s2_to_s1,s3_to_s1,W_eff/c,a/c,r_adr[kg/m3],r_adn[kg/m3],...,r_bdn[kg/m3],WA_RCA[%],WA_NA[%],CI_RCA[%],CI_NA[%],f_c[MPa],e0[10^-3],Ec[GPa],s_1[MPa],e1[10^-3]
0,S2,150,150,30,0.00,0.0,0.40,3.17,2560,2720.0,...,1470.0,2.91,0.56,11.42,3.89,22.85,NaN,NaN,22.85,NaN
1,S2,150,150,30,1.00,0.0,0.40,3.17,2560,2720.0,...,1470.0,2.91,0.56,11.42,3.89,22.85,NaN,NaN,32.19,NaN
2,S2,150,150,30,0.10,0.1,0.40,3.17,2560,2720.0,...,1470.0,2.91,0.56,11.42,3.89,22.85,NaN,NaN,71.19,NaN
3,S2,150,150,30,0.25,0.1,0.40,3.17,2560,2720.0,...,1470.0,2.91,0.56,11.42,3.89,22.85,NaN,NaN,96.40,NaN
4,S2,150,150,30,0.50,0.1,0.40,3.17,2560,2720.0,...,1470.0,2.91,0.56,11.42,3.89,22.85,NaN,NaN,112.02,NaN
5,S2,150,150,30,0.75,0.1,0.40,3.17,2560,2720.0,...,1470.0,2.91,0.56,11.42,3.89,22.85,NaN,NaN,109.26,NaN
6,S2,150,150,30,1.00,0.1,0.40,3.17,2560,2720.0,...,1470.0,2.91,0.56,11.42,3.89,22.85,NaN,NaN,97.32,NaN
7,S2,150,150,50,0.00,0.0,0.37,3.13,2560,2720.0,...,1470.0,2.91,0.56,11.42,3.89,27.74,NaN,NaN,27.74,NaN
8,S2,150,150,50,1.00,0.0,0.37,3.13,2560,2720.0,...,1470.0,2.91,0.56,11.42,3.89,27.74,NaN,NaN,39.93,NaN
9,S2,150,150,50,0.10,0.1,0.37,3.13,2560,2720.0,...,1470.0,2.91,0.56,11.42,3.89,27.74,NaN,NaN,90.39,NaN


#### **Feature Selection using Grey Correlation Analysis**

The paper uses Grey Correlation Analysis to determine important features from the given dataset. 

Using this analysis, the paper determined the following:
- RAC%, weff/c, a/c, S_3 and T can be used for calculating peak stress, peak strain and elastic moduli in axisymmetric load cases
- RAC%, s2_to_s1, s3_to_s1, weff/c and a/c can be used for calculating peak stress in true triaxial load cases

#### **Prediction of the triaxial behavior of RAC using multivariable regression**

**Multiple non-linear regression**

The mathematical form of the polynomial is assumed as follows: <br><br>
$y = \sum_{i=1}^{n}\beta _{i}x_{i} + \sum_{i=1}^{n-1}\sum_{j=i+1}^{n}\beta _{ij}x_{i}x_{j} + \varepsilon $

where y represents the dependent variable (i.e., σ<sub>1</sub>, ε<sub>1</sub> and Ε<sub>c</sub>) of thetriaxial response; x is the independent variable; edenotes the normally distributed error; β<sub>i</sub> and β<sub>ij</sub> are the proportion coefficients of the first-order and the second-order polynomials, respectively.
<br><br>
Using the nonlinear regression analysis, we will determine the coefficients of β<sub>i</sub> and β<sub>ij</sub>

The above regression model uses multi-variable relationship between different features. 

- For axisymmetric triaxial load, the dataset A1 is used to predict peak stress, peak strain and elastic moduli

- For true triaxial load, the dataset A2 is used to predict the peak stress and uses features that differs from axisymmetric cases

Hence, new datasets are created which accomodate multi-variable relationship columns 

**Creating polynomial features for the dataset A1**

In [ ]:
'''
target = 's_1[MPa]'
extra_columns = ['Specimen_Shape', 'D[mm]', 'H[mm]', 'r_adr[kg/m3]', 
    'r_adn[kg/m3]', 'r_bdr[kg/m3]','r_bdn[kg/m3]','WA_RCA[%]','WA_NA[%]',
    'CI_RCA[%]','CI_NA[%]','f_c[MPa]','e0[10^-3]']

d = LocalDatasets(file_path='./dataset_tableA1.csv', 
                  target=target, 
                  extra_columns=extra_columns, 
                  testdata_columns=None,
                  as_frame=True)


poly_A1 = pd.DataFrame([dataA1['RCA%'], dataA1['w_eff/c'], dataA1['a/c'], dataA1['S_3[MPa]'], dataA1['T[deg.C]'], 
    dataA1['RCA%']*dataA1['w_eff/c'],
    dataA1['RCA%']*dataA1['a/c'],
    dataA1['RCA%']*dataA1['S_3[MPa]'],
    dataA1['RCA%']*dataA1['T[deg.C]'],
    dataA1['w_eff/c']*dataA1['a/c'],
    dataA1['w_eff/c']*dataA1['S_3[MPa]'],
    dataA1['w_eff/c']*dataA1['T[deg.C]'],
    dataA1['a/c']*dataA1['S_3[MPa]'],
    dataA1['a/c']*dataA1['T[deg.C]'],
    dataA1['S_3[MPa]']*dataA1['T[deg.C]'],
    dataA1['Ec[GPa]'],dataA1['s_1[MPa]'],dataA1['e1[10^-3]']
    ]).T

poly_A1
'''

"\ntarget = 's_1[MPa]'\nextra_columns = ['Specimen_Shape', 'D[mm]', 'H[mm]', 'r_adr[kg/m3]', \n    'r_adn[kg/m3]', 'r_bdr[kg/m3]','r_bdn[kg/m3]','WA_RCA[%]','WA_NA[%]',\n    'CI_RCA[%]','CI_NA[%]','f_c[MPa]','e0[10^-3]']\n\nd = LocalDatasets(file_path='./dataset_tableA1.csv', \n                  target=target, \n                  extra_columns=extra_columns, \n                  testdata_columns=None,\n                  as_frame=True)\n\n\npoly_A1 = pd.DataFrame([dataA1['RCA%'], dataA1['w_eff/c'], dataA1['a/c'], dataA1['S_3[MPa]'], dataA1['T[deg.C]'], \n    dataA1['RCA%']*dataA1['w_eff/c'],\n    dataA1['RCA%']*dataA1['a/c'],\n    dataA1['RCA%']*dataA1['S_3[MPa]'],\n    dataA1['RCA%']*dataA1['T[deg.C]'],\n    dataA1['w_eff/c']*dataA1['a/c'],\n    dataA1['w_eff/c']*dataA1['S_3[MPa]'],\n    dataA1['w_eff/c']*dataA1['T[deg.C]'],\n    dataA1['a/c']*dataA1['S_3[MPa]'],\n    dataA1['a/c']*dataA1['T[deg.C]'],\n    dataA1['S_3[MPa]']*dataA1['T[deg.C]'],\n    dataA1['Ec[GPa]'],dataA1['s_1[MPa]'],

In [ ]:
SAVEPATH_POLY_1 = '/content/drive/MyDrive/University/2022-23 Senior/Research/MNR_Model/MNR_ModelResults/PolynomialGenerator_DataA1'
mastml_feature_gen1 = Mastml(savepath=SAVEPATH_POLY_1)
savepath_feature_gen1 = mastml_feature_gen1.get_savepath
poly_A1 = PolynomialFeatureGenerator(degree=2, interaction_only=True, include_bias=False).evaluate(dataA1.iloc[:, 3:8], dataA1.iloc[:,18:21], savepath=savepath_feature_gen1)
poly_A1 = pd.concat([poly_A1[0].iloc[:,5:], dataA1['Ec[GPa]'],dataA1['s_1[MPa]'],dataA1['e1[10^-3]']], axis=1)

poly_A1

,x0,x1,x2,x3,x4,x0 x1,x0 x2,x0 x3,x0 x4,x1 x2,x1 x3,x1 x4,x2 x3,x2 x4,x3 x4,Ec[GPa],s_1[MPa],e1[10^-3]
0,0.0,0.40,2.50,0.0,25.0,0.0,0.0,0.0,0.0,1.0000,0.00,10.0,0.00,62.5,0.0,NaN,64.36,2.179
1,0.0,0.40,2.50,8.0,25.0,0.0,0.0,0.0,0.0,1.0000,3.20,10.0,20.00,62.5,200.0,NaN,120.67,10.202
2,0.0,0.40,2.50,16.0,25.0,0.0,0.0,0.0,0.0,1.0000,6.40,10.0,40.00,62.5,400.0,NaN,158.98,15.889
3,0.0,0.40,2.50,24.0,25.0,0.0,0.0,0.0,0.0,1.0000,9.60,10.0,60.00,62.5,600.0,NaN,188.16,19.157
4,0.0,0.40,2.50,32.0,25.0,0.0,0.0,0.0,0.0,1.0000,12.80,10.0,80.00,62.5,800.0,NaN,219.04,24.846
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
182,30.0,0.41,2.15,20.0,500.0,12.3,64.5,600.0,15000.0,0.8815,8.20,205.0,43.00,1075.0,10000.0,NaN,107.83,NaN
183,70.0,0.41,2.15,0.0,500.0,28.7,150.5,0.0,35000.0,0.8815,0.00,205.0,0.00,1075.0,0.0,NaN,7.31,NaN
184,70.0,0.41,2.15,5.0,500.0,28.7,150.5,350.0,35000.0,0.8815,2.05,205.0,10.75,1075.0,2500.0,NaN,53.56,NaN
185,70.0,0.41,2.15,10.0,500.0,28.7,150.5,700.0,35000.0,0.8815,4.10,205.0,21.50,1075.0,5000.0,NaN,76.07,NaN


**Creating polynomial features for the dataset A2**

In [ ]:
'''
poly_A2 = pd.DataFrame([dataA2['RCA%'], dataA2['s2_to_s1'], dataA2['s3_to_s1'], dataA2['W_eff/c'], dataA2['a/c'], 
    dataA2['RCA%']*dataA2['s2_to_s1'],
    dataA2['RCA%']*dataA2['s3_to_s1'],
    dataA2['RCA%']*dataA2['W_eff/c'],
    dataA2['RCA%']*dataA2['a/c'],
    dataA2['s2_to_s1']*dataA2['s3_to_s1'],
    dataA2['s2_to_s1']*dataA2['W_eff/c'],
    dataA2['s2_to_s1']*dataA2['a/c'],
    dataA2['s3_to_s1']*dataA2['W_eff/c'],
    dataA2['s3_to_s1']*dataA2['a/c'],
    dataA2['W_eff/c']*dataA2['a/c'],
    dataA2['Ec[GPa]'],dataA2['s_1[MPa]'],dataA2['e1[10^-3]']
    ]).T
'''
SAVEPATH_POLY_2 = '/content/drive/MyDrive/University/2022-23 Senior/Research/MNR_Model/MNR_ModelResults/PolynomialGenerator_DataA2'
mastml_feature_gen2 = Mastml(savepath=SAVEPATH_POLY_2)
savepath_feature_gen2 = mastml_feature_gen2.get_savepath
poly_A2 = PolynomialFeatureGenerator(degree=2, interaction_only=True, include_bias=False).evaluate(dataA2.iloc[:, 3:8], dataA2.iloc[:,18:21], savepath=savepath_feature_gen2)
poly_A2 = pd.concat([poly_A2[0].iloc[:,5:], dataA2['Ec[GPa]'],dataA2['s_1[MPa]'],dataA2['e1[10^-3]']], axis=1)

poly_A2


,x0,x1,x2,x3,x4,x0 x1,x0 x2,x0 x3,x0 x4,x1 x2,x1 x3,x1 x4,x2 x3,x2 x4,x3 x4,Ec[GPa],s_1[MPa],e1[10^-3]
0,30.0,0.00,0.0,0.40,3.17,0.0,0.0,12.0,95.1,0.000,0.0000,0.0000,0.000,0.000,1.2680,NaN,22.85,NaN
1,30.0,1.00,0.0,0.40,3.17,30.0,0.0,12.0,95.1,0.000,0.4000,3.1700,0.000,0.000,1.2680,NaN,32.19,NaN
2,30.0,0.10,0.1,0.40,3.17,3.0,3.0,12.0,95.1,0.010,0.0400,0.3170,0.040,0.317,1.2680,NaN,71.19,NaN
3,30.0,0.25,0.1,0.40,3.17,7.5,3.0,12.0,95.1,0.025,0.1000,0.7925,0.040,0.317,1.2680,NaN,96.40,NaN
4,30.0,0.50,0.1,0.40,3.17,15.0,3.0,12.0,95.1,0.050,0.2000,1.5850,0.040,0.317,1.2680,NaN,112.02,NaN
5,30.0,0.75,0.1,0.40,3.17,22.5,3.0,12.0,95.1,0.075,0.3000,2.3775,0.040,0.317,1.2680,NaN,109.26,NaN
6,30.0,1.00,0.1,0.40,3.17,30.0,3.0,12.0,95.1,0.100,0.4000,3.1700,0.040,0.317,1.2680,NaN,97.32,NaN
7,50.0,0.00,0.0,0.37,3.13,0.0,0.0,18.5,156.5,0.000,0.0000,0.0000,0.000,0.000,1.1581,NaN,27.74,NaN
8,50.0,1.00,0.0,0.37,3.13,50.0,0.0,18.5,156.5,0.000,0.3700,3.1300,0.000,0.000,1.1581,NaN,39.93,NaN
9,50.0,0.10,0.1,0.37,3.13,5.0,5.0,18.5,156.5,0.010,0.0370,0.3130,0.037,0.313,1.1581,NaN,90.39,NaN


The paper determines peak stress, peak strain and elastic moduli of axisymmetric triaxial load using the same features from the dataset A1. Hence, we will be creating three different models to predict peak stress, peak strain and elastic moduli. <br><br>
Note: I tried to stack all three prediction under one DataFrame to make this process more simpler and this approach works pretty well when I am using Sklearn model directly. However, the dimension errors shows up when I am using KFold cross-validation on the model.

In [ ]:
model_ens = EnsembleModel(model='LinearRegression', n_estimators=5)
metrics = ['r2_score', 'mean_absolute_error', 'root_mean_squared_error']
preprocessor = SklearnPreprocessor(preprocessor='StandardScaler', as_frame=True)

**MNR Model to predict peak stress under axisymmetric triaxial load**

In [ ]:
dataModel1 = poly_A1[~poly_A1['s_1[MPa]'].isnull()]
X1 = dataModel1.iloc[:, 0:15]
y1 = dataModel1['s_1[MPa]']

In [ ]:
SAVEPATH_MODEL1 = '/content/drive/MyDrive/University/2022-23 Senior/Research/MNR_Model/MNR_ModelResults/Model_1'

mastml_instance1 = Mastml(savepath=SAVEPATH_MODEL1)
savepath_model1 = mastml_instance1.get_savepath

In [ ]:
splitter1 = NoSplit()
splitter1.evaluate(X=X1,
                  y=y1, 
                  models=[model_ens],
                  mastml=mastml_instance1,
                  metrics=metrics,
                  preprocessor=preprocessor,
                  plots=['Scatter', 'Histogram', 'Error'],
                  savepath=savepath_model1,
                  parallel_run=True,
                  recalibrate_errors=True,
                  verbosity=3)

**MNR Model to predict peak strain under axisymmetric triaxial load**

In [ ]:
dataModel2 = poly_A1[~poly_A1['e1[10^-3]'].isnull()]
X2 = dataModel2.iloc[:, 0:15]
y2 = dataModel2['e1[10^-3]']

In [ ]:
SAVEPATH_MODEL2 = '/content/drive/MyDrive/University/2022-23 Senior/Research/MNR_Model/MNR_ModelResults/Model_2'

mastml_instance2 = Mastml(savepath=SAVEPATH_MODEL2)
savepath_model2 = mastml_instance2.get_savepath

In [ ]:
splitter2 = NoSplit()
splitter2.evaluate(X=X2,
                  y=y2, 
                  models=[model_ens],
                  mastml=mastml_instance2,
                  metrics=metrics,
                  preprocessor=preprocessor,
                  plots=['Scatter', 'Histogram', 'Error'],
                  savepath=savepath_model2,
                  parallel_run=True,
                  recalibrate_errors=True,
                  verbosity=3)

**MNR Model to predict elastic moduli under axisymmetric triaxial load**

In [ ]:
dataModel3 = poly_A1[~poly_A1['Ec[GPa]'].isnull()]
X3 = dataModel3.iloc[:, 0:15]
y3 = dataModel3['Ec[GPa]']

In [ ]:
SAVEPATH_MODEL3 = '/content/drive/MyDrive/University/2022-23 Senior/Research/MNR_Model/MNR_ModelResults/Model_3'

mastml_instance3 = Mastml(savepath=SAVEPATH_MODEL3)
savepath_model3 = mastml_instance3.get_savepath

In [ ]:
splitter3 = NoSplit()
splitter3.evaluate(X=X3,
                  y=y3, 
                  models=[model_ens],
                  mastml=mastml_instance3,
                  metrics=metrics,
                  preprocessor=preprocessor,
                  plots=['Scatter', 'Histogram', 'Error'],
                  savepath=savepath_model3,
                  parallel_run=True,
                  recalibrate_errors=True,
                  verbosity=3)

**MNR Model to predict peak stress under true triaxial load**

In [ ]:
dataModel4 = poly_A2[~poly_A2['s_1[MPa]'].isnull()]
X4 = dataModel4.iloc[:, 0:15]
y4 = dataModel4['s_1[MPa]']

In [ ]:
SAVEPATH_MODEL4 = '/content/drive/MyDrive/University/2022-23 Senior/Research/MNR_Model/MNR_ModelResults/Model_4'

mastml_instance4 = Mastml(savepath=SAVEPATH_MODEL4)
savepath_model4 = mastml_instance4.get_savepath

In [ ]:
splitter4 = NoSplit()
splitter4.evaluate(X=X4,
                  y=y4, 
                  models=[model_ens],
                  mastml=mastml_instance4,
                  metrics=metrics,
                  preprocessor=preprocessor,
                  plots=['Scatter', 'Histogram', 'Error'],
                  savepath=savepath_model4,
                  parallel_run=True,
                  recalibrate_errors=True,
                  verbosity=3)